In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ML Project/data/train.csv')

train_indices = np.loadtxt('/content/drive/MyDrive/ML Project/data/train_indices.txt', dtype=int)
test_indices = np.loadtxt('/content/drive/MyDrive/ML Project/data/test_indices.txt', dtype=int)

train_df = df.iloc[train_indices]
test_df = df.iloc[test_indices]

<ipython-input-3-a4164eeb9076>:3: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  train_indices = np.loadtxt('/content/drive/MyDrive/ML Project/data/train_indices.txt', dtype=int)
<ipython-input-3-a4164eeb9076>:4: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating p

In [ ]:
train_df.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
1501077,771fda0ff8ff,969f1c4acd66,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,Продам системник,продам системник на запчасти.,50.0,18,2017-03-28,Private,5f1a4dc40d333d5cd20c9831553a61053e83ffb156f692...,2789.0,0.76786
51680,3c296fc97db9,cf345facfe6b,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,Продам раздвижные коньки на девочку 32-36 разм,Продам раздвижные коньки на девочку б/у. Разм....,500.0,30,2017-03-16,Private,659f553a15733bbc034bc2ea321a8aaa39feaa3fb1e410...,2645.0,0.00000
26826,13ecb01f22c1,a4ad6dcc8446,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,Ромашки и др,1фото-ромашка крупноцветковая/\n2фото-пиретрум...,80.0,25,2017-03-16,Private,7f5b347e054800f384d22a9959792ab6da7b1442931d03...,406.0,0.12311
867845,f3a81005ce3d,a41d7a0cd76f,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),"Зимний комплект Bilemi p,98","Продам зимний костюм Bilemi p,98 в отличном со...",1500.0,182,2017-03-26,Private,f17dae532061e4a29e25a44fbd90d40ae894c697d636d5...,94.0,0.80323
137360,bc91d5eb9229,19757fb0047c,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,Новая духовка Weissgauff самоочистка сенсор га...,В магазине 22000руб/\n/\nТак же есть новая тех...,18990.0,422,2017-03-16,Shop,ef7789caae6b81233991c9485cf081bcd6530aa4bd9735...,1852.0,0.00000


Index(['item_id', 'user_id', 'region', 'city', 'parent_category_name',
       'category_name', 'param_1', 'param_2', 'param_3', 'title',
       'description', 'price', 'item_seq_number', 'activation_date',
       'user_type', 'image', 'image_top_1', 'deal_probability'],
      dtype='object')

# Feature Engineering

In [ ]:
def feature_engineering(train_X):
  # 1. category
  train_X['region_city'] = train_X['region'] + '_' + train_X['city']
  train_X['all_category'] = train_X['parent_category_name'] + '_' + train_X['category_name']
  train_X['category_param_1'] = train_X['category_name'].astype(str) + '_' + train_X['param_1'].astype(str)
  train_X['region_category_user'] = train_X['region'] + '_' + train_X['category_name'] + '_' + train_X['user_type']
  train_X['city_category_user'] = train_X['city'] + '_' + train_X['category_name'] + '_' + train_X['user_type']

  group = train_X.groupby('category_name')['price']
  train_X['category_price_mean'] = group.transform('mean')
  train_X['category_price_std'] = group.transform('std')
  train_X['category_price_max'] = group.transform('max')
  # train_X['category_price_min'] = group.transform('min')
  # train_X['category_price_range'] = train_X['category_price_max'] - train_X['category_price_min']
  train_X['category_price_skew'] = group.transform('skew')


  group1 = train_X.groupby('city')['price']
  train_X['city_price_mean'] = group1.transform('mean')
  train_X['city_price_std'] = group1.transform('std')
  train_X['city_price_max'] = group1.transform('max')
  # train_X['city_price_min'] = group1.transform('min')
  # train_X['city_price_range'] = train_X['city_price_max'] - train_X['city_price_min']
  train_X['city_price_skew'] = group1.transform('skew')


  # 2. texts
  train_X['title_length'] = train_X['title'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
  train_X['description_length'] = train_X['description'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
  train_X['title_word_count'] = train_X['title'].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)
  train_X['description_word_count'] = train_X['description'].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

  keywords = ['скидка', 'дешево', 'новый', 'продам', 'купить']
  train_X['title_has_keyword'] = train_X['title'].apply(lambda x: any(kw in str(x).lower() for kw in keywords))
  train_X['description_has_keyword'] = train_X['description'].apply(lambda x: any(kw in str(x).lower() for kw in keywords))

  train_X['title_digit_count'] = train_X['title'].apply(lambda x: sum(c.isdigit() for c in str(x)) if pd.notnull(x) else 0)
  train_X['description_digit_count'] = train_X['description'].apply(lambda x: sum(c.isdigit() for c in str(x)) if pd.notnull(x) else 0)
  train_X['description_newline_count'] = train_X['description'].apply(lambda x: str(x).count('\n') if pd.notnull(x) else 0)

  # 3. price
  train_X['price_log'] = train_X['price'].apply(lambda x: np.log1p(x) if pd.notnull(x) else x)
  custom_bins = [0, 400, 800, 2050, 12000, float('inf')]
  train_X['price_bin'] = pd.cut(train_X['price'], bins=custom_bins, labels=False, right=False)
  train_X['price_bin'] = train_X['price_bin'].fillna(-1).astype(int)

  train_X['price_to_category_mean'] = train_X['price'] / train_X['category_price_mean']
  train_X['price_to_category_max'] = train_X['price'] / train_X['category_price_max']

  # 4. nulls
  for col in ['price','param_1', 'param_2', 'param_3','description']:
    train_X[f'{col}_missing'] = train_X[col].isnull().astype(int)

  return train_X

In [ ]:
tabular_features = [
    'region', 'city', 'parent_category_name',
    'category_name', 'param_1', 'param_2', 'param_3', 'price', 'user_type'
]
text_features = ['title', 'description']

train_X = train_df[tabular_features+text_features]
train_y = train_df['deal_probability']

test_X = test_df[tabular_features+text_features]
test_y = test_df['deal_probability']

In [ ]:
train_X_features = feature_engineering(train_X)

<ipython-input-6-7e58f3eaa9bc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_X['region_city'] = train_X['region'] + '_' + train_X['city']
<ipython-input-6-7e58f3eaa9bc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_X['all_category'] = train_X['parent_category_name'] + '_' + train_X['category_name']
<ipython-input-6-7e58f3eaa9bc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [ ]:
pd.set_option('display.max_columns', None)
train_X_features

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,title,description,region_city,all_category,category_param_1,region_category_user,city_category_user,category_price_mean,category_price_std,category_price_max,category_price_skew,city_price_mean,city_price_std,city_price_max,city_price_skew,title_length,description_length,title_word_count,description_word_count,title_has_keyword,description_has_keyword,title_digit_count,description_digit_count,description_newline_count,price_log,price_bin,price_to_category_mean,price_to_category_max,price_missing,param_1_missing,param_2_missing,param_3_missing,description_missing
1501077,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,50.0,Private,Продам системник,продам системник на запчасти.,Омская область_Омск,Бытовая электроника_Настольные компьютеры,Настольные компьютеры_nan,Омская область_Настольные компьютеры_Private,Омск_Настольные компьютеры_Private,1.285984e+04,3.192384e+04,1500000.0,34.072850,8.485618e+04,2.190840e+06,3.503004e+08,133.726725,16,29,2,4,True,True,0,0,0,3.931826,0,0.003888,3.333333e-05,0,1,1,1,0
51680,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,500.0,Private,Продам раздвижные коньки на девочку 32-36 разм,Продам раздвижные коньки на девочку б/у. Разм....,Башкортостан_Уфа,Хобби и отдых_Спорт и отдых,Спорт и отдых_Зимние виды спорта,Башкортостан_Спорт и отдых_Private,Уфа_Спорт и отдых_Private,1.120145e+04,5.076641e+05,70005500.0,134.569090,2.148687e+05,2.552492e+06,3.700000e+08,104.630644,46,53,7,8,True,True,4,4,0,6.216606,1,0.044637,7.142296e-06,0,0,1,1,0
26826,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,80.0,Private,Ромашки и др,1фото-ромашка крупноцветковая/\n2фото-пиретрум...,Нижегородская область_Дзержинск,Для дома и дачи_Растения,Растения_nan,Нижегородская область_Растения_Private,Дзержинск_Растения_Private,2.328682e+03,2.718157e+04,1000500.0,26.888604,1.191266e+05,1.537133e+06,1.000000e+08,52.994398,12,317,3,27,False,False,0,15,11,4.394449,0,0.034354,7.996002e-05,0,1,1,1,0
867845,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),1500.0,Private,"Зимний комплект Bilemi p,98","Продам зимний костюм Bilemi p,98 в отличном со...",Челябинская область_Миасс,Личные вещи_Детская одежда и обувь,Детская одежда и обувь_Для девочек,Челябинская область_Детская одежда и обувь_Pri...,Миасс_Детская одежда и обувь_Private,7.879266e+03,1.327932e+06,300600400.0,214.378003,1.794778e+05,6.116471e+05,1.400000e+07,8.481999,27,53,4,8,False,True,2,2,0,7.313887,2,0.190373,4.990013e-06,0,0,0,0,0
137360,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,18990.0,Shop,Новая духовка Weissgauff самоочистка сенсор га...,В магазине 22000руб/\n/\nТак же есть новая тех...,Башкортостан_Уфа,Для дома и дачи_Бытовая техника,Бытовая техника_Для кухни,Башкортостан_Бытовая техника_Shop,Уфа_Бытовая техника_Shop,8.064269e+03,5.199713e+04,5001500.0,68.909386,2.148687e+05,2.552492e+06,3.700000e+08,104.630644,50,2043,6,291,False,False,0,47,90,9.851720,4,2.354832,3.796861e-03,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,Ростовская область,Ростов-на-Дону,Услуги,Предложение услуг,"Ремонт, строительство",Сантехника,NaN,NaN,Private,Бригада выполняет работы,"Выполняем прокладку наружного трубопровода,мон...",Ростовская область_Ростов-на-Дону,Услуги_Предложение услуг,"Предложение услуг_Ремонт, строительство",Ростовская область_Предложение услуг_Private,Ростов-на-Дону_Предложение услуг_Private,1.549558e+04,9.602661e+05,100000000.0,101.278974,4.280137e+05,2.701603e+06,2.200000e+08,40.750012,24,113,3,12,False,False,0,0,0,NaN,-1,NaN,NaN,1,0,0,1,0
1414414,Тульская область,Тула,Недвижимость,Квартиры,Сдам,На длительный срок,2,13500.0,Company,"2-к квартира, 46 м², 2/5 эт.",Сдается уютная 2-комнатная

In [ ]:
train_X_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1202739 entries, 1501077 to 121958
Data columns (total 42 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   region                     1202739 non-null  object 
 1   city                       1202739 non-null  object 
 2   parent_category_name       1202739 non-null  object 
 3   category_name              1202739 non-null  object 
 4   param_1                    1153263 non-null  object 
 5   param_2                    679119 non-null   object 
 6   param_3                    512840 non-null   object 
 7   price                      1134146 non-null  float64
 8   user_type                  1202739 non-null  object 
 9   title                      1202739 non-null  object 
 10  description                1109749 non-null  object 
 11  region_city                1202739 non-null  object 
 12  all_category               1202739 non-null  object 
 13  category_par

In [ ]:
train_X['is_train'] =1
test_X['is_train'] = 0
train_test_X = pd.concat([train_X, test_X])

<ipython-input-11-aff4675989bb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_X['is_train'] = 0


In [ ]:
train_test_X

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,title,description,region_city,all_category,category_param_1,region_category_user,city_category_user,category_price_mean,category_price_std,category_price_max,category_price_skew,city_price_mean,city_price_std,city_price_max,city_price_skew,title_length,description_length,title_word_count,description_word_count,title_has_keyword,description_has_keyword,title_digit_count,description_digit_count,description_newline_count,price_log,price_bin,price_to_category_mean,price_to_category_max,price_missing,param_1_missing,param_2_missing,param_3_missing,description_missing,is_train
1501077,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,50.0,Private,Продам системник,продам системник на запчасти.,Омская область_Омск,Бытовая электроника_Настольные компьютеры,Настольные компьютеры_nan,Омская область_Настольные компьютеры_Private,Омск_Настольные компьютеры_Private,12859.837365,3.192384e+04,1500000.0,34.072850,84856.181346,2.190840e+06,350300400.0,133.726725,16.0,29.0,2.0,4.0,True,True,0.0,0.0,0.0,3.931826,0.0,0.003888,0.000033,0.0,1.0,1.0,1.0,0.0,1
51680,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,500.0,Private,Продам раздвижные коньки на девочку 32-36 разм,Продам раздвижные коньки на девочку б/у. Разм....,Башкортостан_Уфа,Хобби и отдых_Спорт и отдых,Спорт и отдых_Зимние виды спорта,Башкортостан_Спорт и отдых_Private,Уфа_Спорт и отдых_Private,11201.452674,5.076641e+05,70005500.0,134.569090,214868.737934,2.552492e+06,370000000.0,104.630644,46.0,53.0,7.0,8.0,True,True,4.0,4.0,0.0,6.216606,1.0,0.044637,0.000007,0.0,0.0,1.0,1.0,0.0,1
26826,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,80.0,Private,Ромашки и др,1фото-ромашка крупноцветковая/\n2фото-пиретрум...,Нижегородская область_Дзержинск,Для дома и дачи_Растения,Растения_nan,Нижегородская область_Растения_Private,Дзержинск_Растения_Private,2328.682280,2.718157e+04,1000500.0,26.888604,119126.624110,1.537133e+06,100000000.0,52.994398,12.0,317.0,3.0,27.0,False,False,0.0,15.0,11.0,4.394449,0.0,0.034354,0.000080,0.0,1.0,1.0,1.0,0.0,1
867845,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),1500.0,Private,"Зимний комплект Bilemi p,98","Продам зимний костюм Bilemi p,98 в отличном со...",Челябинская область_Миасс,Личные вещи_Детская одежда и обувь,Детская одежда и обувь_Для девочек,Челябинская область_Детская одежда и обувь_Pri...,Миасс_Детская одежда и обувь_Private,7879.266072,1.327932e+06,300600400.0,214.378003,179477.819044,6.116471e+05,14000000.0,8.481999,27.0,53.0,4.0,8.0,False,True,2.0,2.0,0.0,7.313887,2.0,0.190373,0.000005,0.0,0.0,0.0,0.0,0.0,1
137360,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,18990.0,Shop,Новая духовка Weissgauff самоочистка сенсор га...,В магазине 22000руб/\n/\nТак же есть новая тех...,Башкортостан_Уфа,Для дома и дачи_Бытовая техника,Бытовая техника_Для кухни,Башкортостан_Бытовая техника_Shop,Уфа_Бытовая техника_Shop,8064.269305,5.199713e+04,5001500.0,68.909386,214868.737934,2.552492e+06,370000000.0,104.630644,50.0,2043.0,6.0,291.0,False,False,0.0,47.0,90.0,9.851720,4.0,2.354832,0.003797,0.0,0.0,0.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186583,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,1000.0,Private,Сабо,"Состояние хорошее,замша натуральная.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
540298,Свердловская область,Екатеринбург,Бытовая электроника,Телефоны,LG,NaN,NaN,5500.0,Private,Lg Leon h 324,"Документы имеются.В хорошем рабочем состоянии,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
train_test_X_featured = feature_engineering(train_test_X)
train_test_X_featured

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,title,description,region_city,all_category,category_param_1,region_category_user,city_category_user,category_price_mean,category_price_std,category_price_max,category_price_skew,city_price_mean,city_price_std,city_price_max,city_price_skew,title_length,description_length,title_word_count,description_word_count,title_has_keyword,description_has_keyword,title_digit_count,description_digit_count,description_newline_count,price_log,price_bin,price_to_category_mean,price_to_category_max,price_missing,param_1_missing,param_2_missing,param_3_missing,description_missing,is_train
1501077,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,50.0,Private,Продам системник,продам системник на запчасти.,Омская область_Омск,Бытовая электроника_Настольные компьютеры,Настольные компьютеры_nan,Омская область_Настольные компьютеры_Private,Омск_Настольные компьютеры_Private,12979.179272,3.030708e+04,1500000.0,33.027322,82009.208014,1.980805e+06,350300400.0,145.003248,16,29,2,4,True,True,0,0,0,3.931826,0,0.003852,3.333333e-05,0,1,1,1,0,1
51680,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,500.0,Private,Продам раздвижные коньки на девочку 32-36 разм,Продам раздвижные коньки на девочку б/у. Разм....,Башкортостан_Уфа,Хобби и отдых_Спорт и отдых,Спорт и отдых_Зимние виды спорта,Башкортостан_Спорт и отдых_Private,Уфа_Спорт и отдых_Private,10372.418862,4.545453e+05,70005500.0,150.221291,209505.665705,2.357666e+06,370000000.0,107.626345,46,53,7,8,True,True,4,4,0,6.216606,1,0.048205,7.142296e-06,0,0,1,1,0,1
26826,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,80.0,Private,Ромашки и др,1фото-ромашка крупноцветковая/\n2фото-пиретрум...,Нижегородская область_Дзержинск,Для дома и дачи_Растения,Растения_nan,Нижегородская область_Растения_Private,Дзержинск_Растения_Private,2259.272940,2.953192e+04,1500000.0,32.812337,113500.432385,1.403977e+06,100000000.0,56.181104,12,317,3,27,False,False,0,15,11,4.394449,0,0.035410,5.333333e-05,0,1,1,1,0,1
867845,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),1500.0,Private,"Зимний комплект Bilemi p,98","Продам зимний костюм Bilemi p,98 в отличном со...",Челябинская область_Миасс,Личные вещи_Детская одежда и обувь,Детская одежда и обувь_Для девочек,Челябинская область_Детская одежда и обувь_Pri...,Миасс_Детская одежда и обувь_Private,9787.546269,1.486987e+06,350150250.0,190.711737,188495.822392,6.875252e+05,14000000.0,9.843327,27,53,4,8,False,True,2,2,0,7.313887,2,0.153256,4.283875e-06,0,0,0,0,0,1
137360,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,18990.0,Shop,Новая духовка Weissgauff самоочистка сенсор га...,В магазине 22000руб/\n/\nТак же есть новая тех...,Башкортостан_Уфа,Для дома и дачи_Бытовая техника,Бытовая техника_Для кухни,Башкортостан_Бытовая техника_Shop,Уфа_Бытовая техника_Shop,7944.266900,4.743349e+04,5001500.0,72.973266,209505.665705,2.357666e+06,370000000.0,107.626345,50,2043,6,291,False,False,0,47,90,9.851720,4,2.390403,3.796861e-03,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186583,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,1000.0,Private,Сабо,"Состояние хорошее,замша натуральная.",Красноярский край_Красноярск,"Личные вещи_Одежда, обувь, аксессуары","Одежда, обувь, аксессуары_Женская одежда","Красноярский край_Одежда, обувь, аксессуары_Pr...","Красноярск_Одежда, обувь, аксессуары_Private",8243.247392,1.160211e+06,400200100.0,280.672500,192017.408979,3.495369e+06,500400400.0,110.248039,4,36,1,3,False,False,0,0,0,6.908755,2,0.121311,2.498750e-06,0,0,0,0,0,0
540298,Свердловская область,Екатеринбург,Бытовая электроника,Телефоны,LG,NaN,NaN,5500.0,Private,Lg Leon h 324,"Документы имеютс

In [ ]:
test_X_featured = train_test_X_featured[train_test_X_featured['is_train']==0]


In [ ]:
test_X_featured

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,title,description,region_city,all_category,category_param_1,region_category_user,city_category_user,category_price_mean,category_price_std,category_price_max,category_price_skew,city_price_mean,city_price_std,city_price_max,city_price_skew,title_length,description_length,title_word_count,description_word_count,title_has_keyword,description_has_keyword,title_digit_count,description_digit_count,description_newline_count,price_log,price_bin,price_to_category_mean,price_to_category_max,price_missing,param_1_missing,param_2_missing,param_3_missing,description_missing,is_train
143309,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,50000.0,Private,Сумка бренд Louis Vuitton,"Сумка коллекционная, в хорошем состоянии! Нату...",Красноярский край_Красноярск,"Личные вещи_Одежда, обувь, аксессуары","Одежда, обувь, аксессуары_Аксессуары","Красноярский край_Одежда, обувь, аксессуары_Pr...","Красноярск_Одежда, обувь, аксессуары_Private",8243.247392,1.160211e+06,400200100.0,280.672500,192017.408979,3.495369e+06,500400400.0,110.248039,25,88,4,13,False,False,0,0,0,10.819798,4,6.065571,1.249375e-04,0,0,1,1,0,0
771572,Ярославская область,Ярославль,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,21950.0,Company,Автокресло Carmate (Ailebebe) Swing Moon Beige,"Посмотреть, примерить и купить автокресло можн...",Ярославская область_Ярославль,Личные вещи_Товары для детей и игрушки,Товары для детей и игрушки_Автомобильные кресла,Ярославская область_Товары для детей и игрушки...,Ярославль_Товары для детей и игрушки_Company,4129.398175,1.353662e+05,35002000.0,206.294640,113287.054455,1.017695e+06,99999999.0,52.658969,46,2044,6,308,False,True,0,37,35,9.996568,4,5.315545,6.271070e-04,0,0,1,1,0,0
1378472,Тюменская область,Тюмень,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Пиджаки и костюмы,52–54 (XXL),2500.0,Private,Пиджак кожанный,"Состояние на 4, торг",Тюменская область_Тюмень,"Личные вещи_Одежда, обувь, аксессуары","Одежда, обувь, аксессуары_Мужская одежда","Тюменская область_Одежда, обувь, аксессуары_Pr...","Тюмень_Одежда, обувь, аксессуары_Private",8243.247392,1.160211e+06,400200100.0,280.672500,276872.987732,3.369922e+06,450000000.0,96.092936,15,20,2,4,False,False,0,1,0,7.824446,3,0.303279,6.246875e-06,0,0,0,0,0,0
846158,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,32,750.0,Company,Сапоги на слякоть котофей нов,Сапожки из невесомого материала ЭВА. Манжет из...,Новосибирская область_Новосибирск,Личные вещи_Детская одежда и обувь,Детская одежда и обувь_Для девочек,Новосибирская область_Детская одежда и обувь_C...,Новосибирск_Детская одежда и обувь_Company,9787.546269,1.486987e+06,350150250.0,190.711737,97112.155237,1.851899e+06,280200100.0,96.038125,29,1015,5,143,False,False,0,24,4,6.621406,1,0.076628,2.141938e-06,0,0,0,0,0,0
84567,Краснодарский край,Новороссийск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),200.0,Private,Юбка,"Юбка женская, размер 42-44, цвет черный, летня...",Краснодарский край_Новороссийск,"Личные вещи_Одежда, обувь, аксессуары","Одежда, обувь, аксессуары_Женская одежда","Краснодарский край_Одежда, обувь, аксессуары_P...","Новороссийск_Одежда, обувь, аксессуары_Private",8243.247392,1.160211e+06,400200100.0,280.672500,340112.099864,2.203736e+06,80000000.0,20.962144,4,74,1,11,False,False,0,4,0,5.303305,0,0.024262,4.997500e-07,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186583,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,1000.0,Private,Сабо,"Состояние хорошее,замша натуральная.",Красноярский край_Красноярск,"Личные вещи_Одежда, обувь, аксессуары","Одежда, обувь, аксессуары_Женская одежда","Красноярский край_Одежда, обувь,

In [ ]:
train_X_features.to_csv('/content/drive/MyDrive/train_X_featured.csv', index=False)

In [ ]:
test_X_featured.to_csv('/content/drive/MyDrive/test_X_featured.csv', index=False)
